In [1]:
# New visualization, uses plotly instead of matplotlib.
import os
import json
import pandas as pd
import plotly.express as px
from data import DataLoader
from utils.visualizations import *

In [2]:
# Load and initialize relevant variables

RESULTS_FILE = "results.json"
YOLO_FILE = "yolo_results.json"

results_path = os.path.join('results', RESULTS_FILE)
yolo_path = os.path.join('results', YOLO_FILE)

def get_results_dataframe():
    with open(results_path) as json_file:
        data_results = json.load(json_file)
    with open(yolo_path) as yolo_file:
        data_yolo = json.load(yolo_file)

    df_agents = pd.json_normalize(data_results['agents'])
    #print(df_agents.shape)
    print(df_agents.head())

    # Convert the items in the list back to dicts (from str)
    for i in range(len(data_yolo)):
        data_yolo[i] = json.loads(data_yolo[i])

    df_yolo = pd.DataFrame.from_records(data_yolo)
    #print(df_yolo.shape)
    #print(df_yolo.head())

    return df_agents, df_yolo



dataloader = DataLoader()
metadata_summary = dataloader.get_metadata_summary()
agents = dataloader.get_entity_ids()
x_waypoints, y_waypoints = zip(*dataloader.get_map()) # List of (x,y).
#waypoints = (x_waypoints, y_waypoints)
x_waypoint_min, x_waypoint_max = min(x_waypoints), max(x_waypoints)
y_waypoint_min, y_waypoint_max = min(x_waypoints), max(x_waypoints)

agent_count = metadata_summary['n_vehicles']

df_agents, df_yolo = get_results_dataframe()
waypoints = {'x': x_waypoints, 'y': y_waypoints}
df_waypoints = pd.DataFrame(waypoints)


           id          x          y     type  crashing  direction  velocity  \
0  camera_1-0 -32.248787  38.429245   person     False   0.000000       0.0   
1    camera_1 -41.668877  48.905540  vehicle     False -90.161186       0.0   
2  camera_1-2 -38.075365  27.281340   person     False   0.000000       0.0   
3    camera_1 -41.668877  48.905540  vehicle     False -90.161186       0.0   
4  camera_2-0 -45.149696  55.715389      car      True   0.000000       0.0   

   detected  matches  timestep intersection.id  intersection.location.x  \
0      True    False         0  intersection_1                    -47.0   
1     False    False         0  intersection_1                    -47.0   
2      True     True         0  intersection_1                    -47.0   
3     False    False         0  intersection_1                    -47.0   
4      True     True         0  intersection_1                    -47.0   

   intersection.location.y  intersection  
0                     21.0     

In [5]:
import plotly.graph_objects as go

fig1 = px.scatter(df_agents, x="x", y="y", animation_frame="timestep", 
                  animation_group="type"
           #size="pop", color="continent", hover_name="country",
           #log_x=True, size_max=55, range_x=[100,100000], range_y=[25,90]
)

fig1.add_traces(go.Scatter(
    x=list(x_waypoints),
    y=list(y_waypoints),
    line=dict(color='grey'),
    mode='markers',
    name="Roads"
))

# Re-order the data: Trick to change "zorder", or depth of markers.
fig1.data = (fig1.data[1], fig1.data[0])

fig1.show()